In [ ]:
# default_exp layers

In [ ]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
# export
import torch
from torch import nn
import torch.nn.functional as F
from functools import partial

norm_types = (
    nn.BatchNorm1d,
    nn.BatchNorm2d,
    nn.BatchNorm3d,
    nn.InstanceNorm1d,
    nn.InstanceNorm2d,
    nn.InstanceNorm3d,
    nn.LayerNorm,
)

<IPython.core.display.Javascript object>

In [ ]:
# export
def requires_grad(m):
    "Check if the first parameter of `m` requires grad or not"
    ps = list(m.parameters())
    return ps[0].requires_grad if len(ps) > 0 else False


def init_default(m, func=nn.init.kaiming_normal_):
    "Initialize `m` weights with `func` and set `bias` to 0."
    if func:
        if hasattr(m, "weight"):
            func(m.weight)
        if hasattr(m, "bias") and hasattr(m.bias, "data"):
            m.bias.data.fill_(0.0)
    return m


def cond_init(m, func):
    "Apply `init_default` to `m` unless it's a batchnorm module"
    if (not isinstance(m, norm_types)) and requires_grad(m):
        init_default(m, func)


def apply_leaf(m, f):
    "Apply `f` to children of `m`."
    c = m.children()
    if isinstance(m, nn.Module):
        f(m)
    for l in c:
        apply_leaf(l, f)


def apply_init(m, func=nn.init.kaiming_normal_):
    "Initialize all non-batchnorm layers of `m` with `func`."
    apply_leaf(m, partial(cond_init, func=func))

<IPython.core.display.Javascript object>

In [ ]:
# export
class AdaptiveConcatPool2d(nn.Module):
    "Layer that concats `AdaptiveAvgPool2d` and `AdaptiveMaxPool2d` from FastAI"

    def __init__(self, size=None):
        super(AdaptiveConcatPool2d, self).__init__()
        self.size = size or 1
        self.ap = nn.AdaptiveAvgPool2d(self.size)
        self.mp = nn.AdaptiveMaxPool2d(self.size)

    def forward(self, x):
        return torch.cat([self.mp(x), self.ap(x)], 1)

<IPython.core.display.Javascript object>

In [ ]:
# export
# mish activation from : https://github.com/fastai/fastai/blob/master/fastai/layers.py#L549
@torch.jit.script
def _mish_jit_fwd(x):
    return x.mul(torch.tanh(F.softplus(x)))


@torch.jit.script
def _mish_jit_bwd(x, grad_output):
    x_sigmoid = torch.sigmoid(x)
    x_tanh_sp = F.softplus(x).tanh()
    return grad_output.mul(x_tanh_sp + x * x_sigmoid * (1 - x_tanh_sp * x_tanh_sp))


class MishJitAutoFn(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        ctx.save_for_backward(x)
        return _mish_jit_fwd(x)

    @staticmethod
    def backward(ctx, grad_output):
        x = ctx.saved_variables[0]
        return _mish_jit_bwd(x, grad_output)


def mish(x):
    return MishJitAutoFn.apply(x)


# Cell
class Mish(nn.Module):
    def __init__(self, inplace=False):
        super(Mish, self).__init__()

    def forward(self, x):
        return MishJitAutoFn.apply(x)

<IPython.core.display.Javascript object>

In [ ]:
# export
def cut_model(model: nn.Module, n: int = -2):
    "cuts `model` layers upto `n`"
    ls = list(model.children())[:n]
    encoder = nn.Sequential(*ls)
    return encoder


def num_features_model(m: nn.Module, in_chs: int = 3):
    "Return the number of output features for `m`."
    m.to("cpu")
    dummy_inp = torch.zeros((32, in_chs, 120, 120))
    dummy_out = m(dummy_inp)
    return dummy_out.size()[1]

<IPython.core.display.Javascript object>

In [ ]:
import torchvision

orig_model = torchvision.models.resnet18(pretrained=False)
model = cut_model(orig_model, -2)

<IPython.core.display.Javascript object>

In [ ]:
num_features_model(model), num_features_model(orig_model)

(512, 1000)

<IPython.core.display.Javascript object>

In [ ]:
# export
def create_head(
    nf: int, n_out: int, lin_ftrs: int = 512, act: nn.Module = nn.ReLU(inplace=True)
):
    "create a custom head for a classifier from FastAI"
    lin_ftrs = [nf, lin_ftrs, n_out]

    pool = AdaptiveConcatPool2d()

    layers = [pool, nn.Flatten()]

    layers += [
        nn.BatchNorm1d(lin_ftrs[0]),
        nn.Dropout(0.25),
        act,
        nn.Linear(lin_ftrs[0], lin_ftrs[1], bias=False),
        nn.BatchNorm1d(lin_ftrs[1]),
        nn.Dropout(0.5),
        act,
        nn.Linear(lin_ftrs[1], lin_ftrs[2], bias=False),
    ]
    return nn.Sequential(*layers)

<IPython.core.display.Javascript object>

In [ ]:
head = create_head(num_features_model(model), n_out=3)
head

Sequential(
  (0): AdaptiveConcatPool2d(
    (ap): AdaptiveAvgPool2d(output_size=1)
    (mp): AdaptiveMaxPool2d(output_size=1)
  )
  (1): Flatten(start_dim=1, end_dim=-1)
  (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Dropout(p=0.25, inplace=False)
  (4): ReLU(inplace=True)
  (5): Linear(in_features=512, out_features=512, bias=False)
  (6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Dropout(p=0.5, inplace=False)
  (8): ReLU(inplace=True)
  (9): Linear(in_features=512, out_features=3, bias=False)
)

<IPython.core.display.Javascript object>

In [ ]:
# hide
from nbdev.export import *

notebook2script()

Converted 00_core.ipynb.
Converted 01_mixmethods.ipynb.
Converted 02_losses.ipynb.
Converted 03_layers.ipynb.
Converted 03a_networks.ipynb.
Converted 04_optimizers_schedules.ipynb.
Converted 05_lightning.core.ipynb.
Converted 05a_lightning.callbacks.ipynb.
Converted 06_fastai.core.ipynb.
Converted index.ipynb.


<IPython.core.display.Javascript object>